# Setup

Make sure this notebook works well in both python 2 and 3, import a few common modules, ensure MatplotLib plots figures inline

In [462]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # for retina screens
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

Configure notebook to display all results in cell

In [463]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Revert to the last line of output only
# InteractiveShell.ast_node_interactivity = "last_expr"

# Load Data

In [464]:
import pandas as pd

houses_train = pd.read_csv('../Data/encoded.houses.train.csv')
houses_test = pd.read_csv('../Data/encoded.houses.test.csv')

In [465]:
print("houses_train dimensions: {}".format(houses_train.shape))
print("houses_test dimensions: {}".format(houses_test.shape))

houses_train dimensions: (1460, 465)
houses_test dimensions: (1459, 465)


In [466]:
pd.set_option("display.max_columns", 328)
houses_train.head(3)

,MSSubClass_lev_x.120,MSSubClass_lev_x.160,MSSubClass_lev_x.180,MSSubClass_lev_x.190,MSSubClass_lev_x.20,MSSubClass_lev_x.30,MSSubClass_lev_x.40,MSSubClass_lev_x.45,MSSubClass_lev_x.50,MSSubClass_lev_x.60,MSSubClass_lev_x.70,MSSubClass_lev_x.75,MSSubClass_lev_x.80,MSSubClass_lev_x.85,MSSubClass_lev_x.90,MSZoning_lev_x.C..all.,MSZoning_lev_x.FV,MSZoning_lev_x.RH,MSZoning_lev_x.RL,MSZoning_lev_x.RM,LotFrontage_clean,LotArea_clean,Street_lev_x.Grvl,Street_lev_x.Pave,Alley_lev_x.Grvl,Alley_lev_x.NA,Alley_lev_x.Pave,LotShape_lev_x.IR1,LotShape_lev_x.IR2,LotShape_lev_x.IR3,LotShape_lev_x.Reg,LandContour_lev_x.Bnk,LandContour_lev_x.HLS,LandContour_lev_x.Low,LandContour_lev_x.Lvl,Utilities_lev_x.AllPub,Utilities_lev_x.NoSeWa,LotConfig_lev_x.Corner,LotConfig_lev_x.CulDSac,LotConfig_lev_x.FR2,LotConfig_lev_x.FR3,LotConfig_lev_x.Inside,LandSlope_lev_x.Gtl,LandSlope_lev_x.Mod,LandSlope_lev_x.Sev,Neighborhood_lev_x.Blmngtn,Neighborhood_lev_x.Blueste,Neighborhood_lev_x.BrDale,Neighborhood_lev_x.BrkSide,Neighborhood_lev_x.ClearCr,Neighborhood_lev_x.CollgCr,Neighborhood_lev_x.Crawfor,Neighborhood_lev_x.Edwards,Neighborhood_lev_x.Gilbert,Neighborhood_lev_x.IDOTRR,Neighborhood_lev_x.MeadowV,Neighborhood_lev_x.Mitchel,Neighborhood_lev_x.NAmes,Neighborhood_lev_x.NoRidge,Neighborhood_lev_x.NPkVill,Neighborhood_lev_x.NridgHt,Neighborhood_lev_x.NWAmes,Neighborhood_lev_x.OldTown,Neighborhood_lev_x.Sawyer,Neighborhood_lev_x.SawyerW,Neighborhood_lev_x.Somerst,Neighborhood_lev_x.StoneBr,Neighborhood_lev_x.SWISU,Neighborhood_lev_x.Timber,Neighborhood_lev_x.Veenker,Condition1_lev_x.Artery,Condition1_lev_x.Feedr,Condition1_lev_x.Norm,Condition1_lev_x.PosA,Condition1_lev_x.PosN,Condition1_lev_x.RRAe,Condition1_lev_x.RRAn,Condition1_lev_x.RRNe,Condition1_lev_x.RRNn,Condition2_lev_x.Artery,Condition2_lev_x.Feedr,Condition2_lev_x.Norm,Condition2_lev_x.PosA,Condition2_lev_x.PosN,Condition2_lev_x.RRAe,Condition2_lev_x.RRAn,Condition2_lev_x.RRNn,BldgType_lev_x.1Fam,BldgType_lev_x.2fmCon,BldgType_lev_x.Duplex,BldgType_lev_x.Twnhs,BldgType_lev_x.TwnhsE,HouseStyle_lev_x.1.5Fin,HouseStyle_lev_x.1.5Unf,HouseStyle_lev_x.1Story,HouseStyle_lev_x.2.5Fin,HouseStyle_lev_x.2.5Unf,HouseStyle_lev_x.2Story,HouseStyle_lev_x.SFoyer,HouseStyle_lev_x.SLvl,OverallQual_lev_x.1,OverallQual_lev_x.10,OverallQual_lev_x.2,OverallQual_lev_x.3,OverallQual_lev_x.4,OverallQual_lev_x.5,OverallQual_lev_x.6,OverallQual_lev_x.7,OverallQual_lev_x.8,OverallQual_lev_x.9,OverallCond_lev_x.1,OverallCond_lev_x.2,OverallCond_lev_x.3,OverallCond_lev_x.4,OverallCond_lev_x.5,OverallCond_lev_x.6,OverallCond_lev_x.7,OverallCond_lev_x.8,OverallCond_lev_x.9,YearBuilt_clean,YearRemodAdd_clean,RoofStyle_lev_x.Flat,RoofStyle_lev_x.Gable,RoofStyle_lev_x.Gambrel,RoofStyle_lev_x.Hip,RoofStyle_lev_x.Mansard,RoofStyle_lev_x.Shed,RoofMatl_lev_x.ClyTile,RoofMatl_lev_x.CompShg,RoofMatl_lev_x.Membran,RoofMatl_lev_x.Metal,RoofMatl_lev_x.Roll,RoofMatl_lev_x.Tar.Grv,RoofMatl_lev_x.WdShake,RoofMatl_lev_x.WdShngl,Exterior1st_lev_x.AsbShng,Exterior1st_lev_x.AsphShn,Exterior1st_lev_x.BrkComm,Exterior1st_lev_x.BrkFace,Exterior1st_lev_x.CBlock,Exterior1st_lev_x.CemntBd,Exterior1st_lev_x.HdBoard,Exterior1st_lev_x.ImStucc,Exterior1st_lev_x.MetalSd,Exterior1st_lev_x.Plywood,Exterior1st_lev_x.Stone,Exterior1st_lev_x.Stucco,Exterior1st_lev_x.VinylSd,Exterior1st_lev_x.Wd.Sdng,Exterior1st_lev_x.WdShing,Exterior2nd_lev_x.AsbShng,Exterior2nd_lev_x.AsphShn,Exterior2nd_lev_x.Brk.Cmn,Exterior2nd_lev_x.BrkFace,Exterior2nd_lev_x.CBlock,Exterior2nd_lev_x.CmentBd,Exterior2nd_lev_x.HdBoard,Exterior2nd_lev_x.ImStucc,Exterior2nd_lev_x.MetalSd,Exterior2nd_lev_x.Other,Exterior2nd_lev_x.Plywood,Exterior2nd_lev_x.Stone,Exterior2nd_lev_x.Stucco,Exterior2nd_lev_x.VinylSd,...,PoolQC_lev_x.Ex,PoolQC_lev_x.Fa,PoolQC_lev_x.Gd,PoolQC_lev_x.NA,Fence_lev_x.GdPrv,Fence_lev_x.GdWo,Fence_lev_x.MnPrv,Fence_lev_x.MnWw,Fence_lev_x.NA,MiscFeature_lev_x.Gar2,MiscFeature_lev_x.NA,MiscFeature_lev_x.Othr,MiscFeature_lev_x.Shed,MiscFeature_lev_x.TenC,MiscVal_clean,MoSold_cl

In [467]:
houses_train.info()
houses_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 465 entries, MSSubClass_lev_x.120 to SalePrice
dtypes: float64(465)
memory usage: 5.2 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Columns: 465 entries, MSSubClass_lev_x.120 to SalePrice
dtypes: float64(429), int64(36)
memory usage: 5.2 MB


Delete first column

In [468]:
# houses_train.drop('Unnamed: 0', axis=1, inplace=True, errors='raise')
# houses_test.drop('Unnamed: 0', axis=1, inplace=True, errors='raise')

# Run xgboost

### Create private training & test set

In [469]:
from sklearn.model_selection import train_test_split

seed = 10
test_ratio = 0.2

X = houses_train.loc[:, houses_train.columns != "SalePrice"].values # convert to np.array
y = houses_train.loc[:, houses_train.columns == "SalePrice"].values

# Take log of SalePrice
y = np.log(y + 1).ravel() # convert to 1D array for model fit (xxx, )


In [470]:
X_pr_train, X_pr_test, y_pr_train, y_pr_test = train_test_split(X, y, test_size=test_ratio, random_state=seed)

In [471]:
print(len(X_pr_train), "train +", len(X_pr_test), "test")

1168 train + 292 test


### Fit Model

In [ ]:
from xgboost import XGBRegressor

xgb_clf = XGBRegressor(max_depth=3, 
                        learning_rate=0.1, 
                        n_estimators=1000, # Number of boosted trees to fit
                        silent=False, # print messages while running 
                        objective='reg:linear', 
                        booster='gbtree', # Specify which booster to use: gbtree, gblinear or dart
                        #for dart see http://xgboost.readthedocs.io/en/latest/tutorials/dart.html 
                        n_jobs=-1, # Number of parallel threads used to run xgboost. (replaces nthread)
                        gamma=0,  # Minimum loss reduction required to make a further partition on a leaf node of the tree.
                        min_child_weight=1, # Minimum sum of instance weight(hessian) needed in a child
                        max_delta_step=0, # Maximum delta step we allow each tree’s weight estimation to be
                        subsample=1, # Subsample ratio of the training instance
                        colsample_bytree=1, # Subsample ratio of columns when constructing each tree
                        colsample_bylevel=1, # Subsample ratio of columns for each split, in each level
                        reg_alpha=0, # L1 regularization term on weights
                        reg_lambda=1, # L2 regularization term on weights
                        scale_pos_weight=1, # Balancing of positive and negative weights
                        base_score=0.5, # The initial prediction score of all instances, global bias
                        random_state=743, 
                        missing=None) # Value in the data which needs to be present as a missing value. If None, defaults to np.nan



In [ ]:
xgb_clf.fit(X_pr_train, y_pr_train)

In [ ]:
# make predictions for test data

y_pr_pred = xgb_clf.predict(X_pr_test)

Evaluate predictions

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_pr_test, y_pr_pred)
rmse = np.sqrt(mse)
rmse


In [ ]:
y_pr_test[1:5]
y_pr_pred[1:5]

### Model Tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

xgb_params = {'max_depth': 3,
              'learning_rate': 0.1, 
              'n_estimators': 100, 
              'objective': 'reg:linear'}

param_grid = {'max_depth': [3, 4], 
              'learning_rate': [0.01, 0.03, 0.05, 0.07], 
              'n_estimators': [2000, 2500, 3000], # Number of boosted trees to fit
              'objective': ['reg:linear'], 
              'booster': ['gbtree', 'gblinear' 'dart'], # Specify which booster to use: gbtree, gblinear or dart
              'n_jobs': [-1], # Number of parallel threads used to run xgboost. (replaces nthread)
              'gamma': [0],  # Minimum loss reduction required to make a further partition on a leaf node of the tree.
              'min_child_weight': [1], # Minimum sum of instance weight(hessian) needed in a child
              'max_delta_step': [0], # Maximum delta step we allow each tree’s weight estimation to be
              'subsample': [1], # Subsample ratio of the training instance
              'colsample_bytree': [0.5, 0.6, 0.7], # Subsample ratio of columns when constructing each tree
              'colsample_bylevel': [0.2, 0.3, 0.4], # Subsample ratio of columns for each split, in each level
              'reg_alpha': [0], # L1 regularization term on weights
              'reg_lambda': [1], # L2 regularization term on weights
              'scale_pos_weight': [1], # Balancing of positive and negative weights
              'base_score': [0.5], # The initial prediction score of all instances, global bias
              'silent': [True],
              'random_state': [10]}

optimized_xgb_clf = GridSearchCV(XGBRegressor(**xgb_params), # scikit-learn estimator interface 
                                 param_grid = param_grid, # Dictionary with parameters names (string) as keys
                                 scoring="neg_mean_squared_error", # controls what metric they apply to the estimators evaluated
                                 n_jobs=-1, # If True, the data is assumed to be identically distributed across the folds,
                                 iid=True, 
                                 refit=True, # Refit an estimator using the best found parameters (best_estimator_)
                                 cv=2, #integer, to specify the number of folds in a (Stratified)KFold. None -> default 3-fold cross validation
                                 verbose=10, # the higher, the more messages
                                 pre_dispatch="2*n_jobs", # number of jobs that get dispatched during parallel execution
                                 error_score="raise", 
                                 return_train_score=False) #If False, the cv_results_ attribute will not include training scores

Inspect the grid

In [ ]:
optimized_xgb_clf

Run grid tuning

In [ ]:
optimized_xgb_clf.fit(X_pr_train, y_pr_train)

The best combination of parameters is:

In [ ]:
optimized_xgb_clf.best_params_

In [ ]:
## The best score is
optimized_xgb_clf.best_score_ # that's the training score so not meaningful

In [ ]:
# optimized_xgb_clf.cv_results_

In [ ]:
# make predictions for test data

y_pr_pred = optimized_xgb_clf.predict(X_pr_test)

Evaluate predictions

In [ ]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_pr_test, y_pr_pred)
rmse = np.sqrt(mse)
rmse


In [ ]:
y_pr_test[1:5]
y_pr_pred[1:5]